In [1]:
import cobra
from os.path import join
    
# Load a cobra model from anywhere you like
data_dir = "/Users/lizrad/Documents/Vibrio_folder/"
model=cobra.io.read_sbml_model(join(data_dir, "iVnat_bigg.xml"))



In [2]:
# generate a json file to load into Escher
cobra.io.save_json_model(model, 'iVnat_newbigg.json')

In [6]:
from escher import Builder
b = Builder(map_name='e_coli_core.Core metabolism')
b.display_in_browser()


Note: You must interrupt the kernel to end this command

Serving to http://127.0.0.1:7655/
[Ctrl-C to exit from terminal, or Ctrl-M i i to interrupt notebook kernel]


127.0.0.1 - - [06/Sep/2018 14:37:58] "GET / HTTP/1.1" 200 -



stopping Server...


In [5]:
maps = Builder(model=model)#map_name='e_coli_core.Core metabolism')

maps.display_in_browser()


Note: You must interrupt the kernel to end this command

Serving to http://127.0.0.1:7655/
[Ctrl-C to exit from terminal, or Ctrl-M i i to interrupt notebook kernel]


127.0.0.1 - - [06/Sep/2018 14:37:37] "GET / HTTP/1.1" 200 -



stopping Server...


In [ ]:
model_modified = model.copy()
# for example, delete a reaction
#model_modified.reactions.GAPD.delete()